<a href="https://colab.research.google.com/github/maryamelnahas/CSEN711-MS2-Cryptography-and-Encryption/blob/main/CSEN711_MS2_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>